In [2]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view)
import vis.fields
import matplotlib.cm as cm
import time

In [3]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [4]:
rod_length = 0.1756505993328395
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('regular_torus_1', thickness, width, weaving_dir)
#  [ ear 396, ear 412, nose 497]

0.058550199777613166 0.004098513984432921


In [5]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

### No smoothing

In [6]:
# # with so(): curved_linkage = initialize_linkage(surface_path = None, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
# # curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
# # curved_save_tgt_joint_pos = curved_linkage.jointPositions()
# curved_linkage = pickle.load(gzip.open('mega_monster_optimization_diagram_results/', 'r'))
# curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
# # set_surface_view_options(curved_linkage_view, linkage_color='lightgreen')
# curved_linkage_view.show()

In [11]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), …

In [8]:
curved_linkage.setExtendedDoFsPSRL(np.load(''))

In [9]:
def eqm_callback(prob, i):
    curved_linkage_view.update()

In [10]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

0	20.1228	131.2	131.2	1	1
1	16.1707	79.5876	79.5876	1	1
2	12.7987	50.5771	50.5771	1	1
3	9.96526	33.5219	33.5219	1	1
4	7.63404	23.456	23.456	1	1
5	5.69947	16.8376	16.8376	1	1
6	4.07128	11.8759	11.8759	1	1
7	2.6987	7.89355	7.89355	1	1
8	1.61085	4.81085	4.81085	1	1
9	0.852912	2.71438	2.71438	1	1
10	0.459773	7.28032	7.28032	1	1
11	0.407946	3.73283	3.73283	1	1
12	0.370034	2.08488	2.08488	1	1
13	0.332475	1.49691	1.49691	1	1
14	0.284118	1.25015	1.25015	1	1
15	0.222681	1.04278	1.04278	1	1
16	0.159551	0.793759	0.793759	1	1
17	0.10976	0.533055	0.533055	1	1
18	0.0788803	0.32935	0.32935	0.0625	1
19	0.078447	0.665006	0.665006	0.25	1
20	0.0766488	0.435743	0.435743	1	1
21	0.0696336	0.250212	0.250212	1	1
22	0.0625258	0.201826	0.201826	1	1
23	0.0566407	0.334918	0.334918	1	1
24	0.0558045	0.116498	0.116498	1	1
25	0.0548225	0.103372	0.103372	1	1
26	0.0533546	0.0895471	0.0895471	1	1
27	0.0513522	0.0850402	0.0850402	1	1
28	0.0505492	0.067983	0.067983	1	1
29	0.0492937	0.120076	0.120076	1	1
30	0.0480899	0.682

In [15]:
import pickle
import gzip
pickle.dump(curved_linkage, gzip.open('{}_stage_2_20210108_crazy.pkl.gz'.format(io.MODEL_NAME), 'w'))